### Summarize outcomes of NODEBNGM

In [1]:
# imports
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import os
import seaborn as sns
import re
import math
from matplotlib.colors import LinearSegmentedColormap

from plot_functions import *

In [2]:
# load folder directory of batch
datasets = [
    'miaSim',
    'miaSim_noise_0-005',
    'miaSim_noise_0-01',
    'miaSim_noise_0-02',
    'miaSim_noise_0-04'.
    '3DLV',
    'VanderPol',
    'VanderPol_noise_0-1',
    'VanderPol_noise_0-2',
    'VanderPol_noise_0-5',
    'VanderPol_noise_1',
    'donorA_ALR',
    'donorB_ALR',
    'male_ALR',
    'female_ALR', 
    'Silverman_all_ALR',
    'Silverman_daily_ALR',
    'Silverman_hourly_ALR',
    'Bucci_ALR',
    'BioTIME_study_339_Genus_10',
    'BioTIME_study_339_Species_15',
    'BioTIME_study_363_Genus_10',
    'BioTIME_study_363_Species_15',
    'BioTIME_study_39_Genus_10',
    'BioTIME_study_39_Species_15',
    'BioTIME_study_478_Genus_10',
    'BioTIME_study_478_Species_15',
    'Ushio'
    ]

output_dir_raw = f"C:/Users/Maria/Documents/Masterstudium/Masterarbeit/MScThesis/R/NODEBNGM/output/"

data_dir_comp = "C:/Users/Maria/Documents/Masterstudium/Masterarbeit/MScThesis/Python/ALR_transformation/ALR_transformed_data/"
data_dir_absolute = "C:/Users/Maria/Documents/Masterstudium/Masterarbeit/MScThesis/explore/data/final_datasets/"

# create color map and load color palette for plots
cmap = LinearSegmentedColormap.from_list("white_to_green", ["white", "darkgreen"])
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Step 1: Calculate weighted effects matrix for all runs
weighted effects = (sign(effects)*weights)

In [3]:
for dataset in datasets:
    # dataset = "female_ALR"
    out_dir = output_dir_raw + "out_" + dataset + "/"

    # get list of available runs for the given dataset
    runs = []
    for file in [s for s in os.listdir(out_dir) if dataset in s]:
        runs.append(file)

    # specify dimensions of the plot
    n_runs = len(runs)

    # check if runs for these specifications are available
    if n_runs > 0:
        for run in runs:
            out_run = out_dir + run
            
            if os.path.exists(f"{out_run}/effectsMat.csv"):
                # read data files
                df_effects_tmp = pd.read_csv(f"{out_run}/effectsMat.csv", header=[0])
                df_weights_tmp = pd.read_csv(f"{out_run}/weightsMat.csv", header=[0])
                n_taxa = df_effects_tmp.shape[0]

                # load and save sign of effectsMat
                df_effects_sign_tmp = np.sign(df_effects_tmp)
                df_effects_sign_tmp.to_csv(f'{out_run}/effectsMat_sign.csv', index=False)

                # calculate weighted effects
                df_weighted_effects_tmp = df_effects_sign_tmp * df_weights_tmp
                df_weighted_effects_tmp.to_csv(f'{out_run}/weighted_effectsMat.csv', index=False)

# Step 2: Create summary plots containing all plots of all runs for one dataset

## Summarize all heatmaps of interaction matrices

In [4]:
for dataset in datasets:
    # dataset = datasets[10]
    print(dataset)
    Mat = "weighted_effectsMat" # "effectsMat", "weightsMat"
    # dataset = datasets[1]
    out_dir = output_dir_raw + "out_" + dataset + "/"

    # get list of available runs for the given dataset
    runs = []
    for file in [s for s in os.listdir(out_dir) if dataset in s]:
        m = re.search(r"run(\d{2})", file)
        if m:
            runs.append(m.group(1))

    # specify dimensions of the plot
    n_runs = len(runs)

    # check if runs for these specifications are available
    if n_runs > 0 and n_runs < 10:
        # specify dimensions of the plot
        n_row = 3
        n_col = 3

        # make plot
        fig, axs = plt.subplots(n_row, n_col)
        # fig.set_figwidth(4*n_col)
        # fig.set_figheight(2*n_row)
        fig.suptitle(f'{dataset}, {Mat}', y=1.0, fontsize = 20)

        y = 0

        for run in runs:
            out_run = out_dir + "out_" + dataset + f"_run{run}"
            # print(out_run)
            
            if os.path.exists(f"{out_run}/{Mat}.csv"):
                # read data files
                df_model_coeffs_tmp = pd.read_csv(f"{out_run}/{Mat}.csv", header=[0])
                n_taxa = df_model_coeffs_tmp.shape[0]
                
                fig.set_figwidth(max(math.ceil(20/n_taxa), 4) * n_taxa)
                fig.set_figheight(max(math.ceil(15/n_taxa), 3) * n_taxa)
                # Decrease the distance between subplots
                fig.tight_layout(pad=1.0, w_pad=0.1, h_pad=0.1)

                # make heatmap
                if Mat == "weightsMat": # scale colors only for positive values
                    sns.heatmap(round(df_model_coeffs_tmp,2), annot=True, fmt="", cmap=cmap, vmin=0, ax=axs[int(y/n_col), y%n_col],
                            xticklabels=False)
                else:
                    sns.heatmap(round(df_model_coeffs_tmp,2), annot=True, fmt="", cmap="vlag", center=0, ax=axs[int(y/n_col), y%n_col],
                            xticklabels=False)
                    
                axs[int(y/n_col), y%n_col].xaxis.tick_top()
                axs[int(y/n_col), y%n_col].set_title(f"run{run}")
                y += 1

        fig.tight_layout(pad=1.0)
        plt.yticks(rotation=0)

        # save plot
        plt.savefig(f'{out_dir}/summary_{Mat}_heatmap.pdf',
                    bbox_inches='tight', dpi = 300)
        plt.close()
        # plt.show()

miaSim_noise_0-005
miaSim_noise_0-01
miaSim_noise_0-02
miaSim_noise_0-04


## Summarize all fits in one plot

In [5]:
for dataset in datasets:
    # dataset = datasets[0]
    print(dataset)
    out_dir = output_dir_raw + "out_" + dataset + "/"

    # get number of taxa
    df = pd.read_csv(f'{out_dir}/out_{dataset}_run05/Names_xi.csv')
    n_taxa = len(df.index)

    # get list of available runs for the given dataset
    runs = []
    for file in [s for s in os.listdir(out_dir) if dataset[0] in s]:
        m = re.search(r"run(\d{2})", file)
        if m:
            runs.append(m.group(1))

    # specify dimensions of the plot
    n_runs = len(runs)

    # check if runs for these specifications are available
    if n_runs > 0 and n_runs < 10:

        # specify dimensions of the plot
        n_row = n_runs
        n_col = n_taxa

        # make plot
        fig, axs = plt.subplots(n_row, n_col)
        fig.set_figwidth(3.5*n_col)
        fig.set_figheight(2*n_row)
        fig.suptitle(dataset, y=1.0, fontsize = 16)
        fig.tight_layout(pad = 2)

        y = 0

        for run in runs:
            out_run = out_dir + "out_" + dataset + f"_run{run}"
            
            if os.path.exists(f"{out_run}/{Mat}.csv"):
                # read data files
                df_pred = pd.read_csv(f"{out_run}/Yhat.csv", header=[0])
                df_data_obs = pd.read_csv(f"{out_run}/TS_{dataset}.csv", header=[0])

                # convert files to numpy array
                data_obs = np.array(df_data_obs)
                pred = np.array(df_pred)

                for taxon in np.arange(n_taxa):
                    # make plot
                    axs[int(y/n_col), (taxon)].plot(data_obs[:,0], pred[:,(taxon)], label = "fit")
                    axs[int(y/n_col), (taxon)].plot(data_obs[:,0], data_obs[:,(taxon+1)], label = "data", linewidth = 0.7, linestyle = '--', color = colors[1])
                    axs[int(y/n_col), (taxon)].set_title(df_data_obs.columns[(taxon+1)])
                    
                    y += 1
                axs[int((y-n_taxa)/n_col), 0].annotate(f"run {run}", xy=(0, 0.5), 
                                                    xytext=(-axs[int((y-n_taxa)/n_col), 0].yaxis.labelpad - 5, 0),
                                                    xycoords=axs[int((y-n_taxa)/n_col), 0].yaxis.label, textcoords='offset points',
                                                    size='large', ha='right', va='center')
                
                # add one legend for all polts (in the lower center)
                handles, labels = axs[0,0].get_legend_handles_labels()
                fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.03),
                            fancybox=True, shadow=False, ncol = 4, fontsize = 12)
        # save plots in one file
        plt.savefig(f'{out_dir}/summary_prediction_fits.pdf',
                    bbox_inches='tight', dpi = 300)
        # save plots in one file
        plt.savefig(f'{out_dir}/summary_prediction_fits.png',
                    bbox_inches='tight', dpi = 300)
        plt.close()

miaSim_noise_0-005
miaSim_noise_0-01
miaSim_noise_0-02
miaSim_noise_0-04


# Step 3: Calculate mean over effects

## Mean over all weightsMats

In [6]:
for dataset in datasets:
    # dataset = datasets[1]
    print(dataset)
    for Mat in ["weightsMat", "weighted_effectsMat"]: # "effectsMat", "weightsMat"
        # Mat = "weightsMat"

        out_dir = output_dir_raw + "out_" + dataset

        # get number of taxa
        df = pd.read_csv(f'{out_dir}/out_{dataset}_run05/Names_xi.csv')
        n_taxa = len(df.index)

        # get list of available runs for the given dataset
        runs = []
        for file in [s for s in os.listdir(out_dir) if dataset in s and "." not in s]:
            runs.append(file)

        # number of available runs
        n_runs = len(runs)

        # check if runs for these specifications are available
        if n_runs > 0:
            
            df_model_coeffs_all = []
            count = 0

            for run in runs:
                # run_nr = re.search(r"run(\d{2})", run).group(1)
                out_run = out_dir + "/" + run
                if os.path.exists(f"{out_run}/{Mat}.csv"):
                    df_model_coeffs_tmp = pd.read_csv(f"{out_run}/{Mat}.csv", header=[0])
                    # print(df_model_coeffs_tmp)
                    df_model_coeffs_all.append(df_model_coeffs_tmp.to_numpy())
        
        if len(df_model_coeffs_all) > 0:
            # calculate mean over all coeff matrices
            mean_array = np.mean(df_model_coeffs_all, axis=0)
            # and save as csv file
            pd.DataFrame(mean_array).to_csv(f'{out_dir}/weighted_effectsMat_mean.csv', index=False)
            
            # make plot
            fig, ax = plt.subplots()
            plot_heatmap(matrix_A = np.around(mean_array, 2), n_taxa = n_taxa, ax = ax, fig=fig,
                        title = f"{dataset}, mean over {Mat}", colnames=df_model_coeffs_tmp.columns,
                        Mat=Mat)
            
            # save plot
            plt.savefig(f'{out_dir}/mean_{Mat}_heatmap.pdf',
                        bbox_inches='tight', dpi = 300)
            plt.close()
            # plt.show()

miaSim_noise_0-005
miaSim_noise_0-01
miaSim_noise_0-02
miaSim_noise_0-04


## Count times of effect > 0 in each cell

In [7]:
# Count the non-zero occurrences for each cell
non_zero_counts = np.count_nonzero(df_model_coeffs_all, axis=0)

print(non_zero_counts)

[[6 9 8 4]
 [7 9 6 7]
 [4 2 9 9]
 [8 3 9 7]]
